In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import GroupKFold
from functions_refactor import RAdam
from functions_cnn import *

In [2]:
type_ = '2JHN'
num_workers = 4
batch_size = 512
clip = 2
n_epochs = 50
model_struct = CNN
root_dir ="../Data/full-images-2jhn/Image_2JHN/"

In [3]:
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
train_ids,test_ids = process_data(train_df,test_df,type_)

/home/will/Desktop/kaggle/QM/Code/functions_cnn.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  train_ids['file_name'] = train_ids.molecule_name.str.cat(train_ids.id.astype(str),sep='_')
/home/will/Desktop/kaggle/QM/Code/functions_cnn.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_ids['file_name'] = test_ids.molecule_name.str.cat(test_ids.id.astype(str),sep='_')


In [4]:
n_epochs =2 

In [7]:
group_kfold = GroupKFold(n_splits=5)
for n_fold, (train_idx, valid_idx) in enumerate(group_kfold.split(train_ids,groups=train_ids['molecule_name'])):
    print('\nstart fold: {}'.format(n_fold))
    
    # set-up data
    train_dl = train_ids.iloc[train_idx]
    valid_dl = train_ids.iloc[valid_idx]
    train_dl = CustomImageDataset(train_dl, root_dir, transform=None, IsTrain=True)
    valid_dl = CustomImageDataset(valid_dl, root_dir, transform=None, IsTrain=True)
    test_dl = CustomImageDataset(test_ids, root_dir, transform=None, IsTrain=False)
    train_dl = torch.utils.data.DataLoader(train_dl,batch_size=batch_size,shuffle=True,num_workers=num_workers)
    valid_dl = torch.utils.data.DataLoader(valid_dl,batch_size=batch_size,shuffle=False,num_workers=num_workers)
    test_dl = torch.utils.data.DataLoader(test_dl,batch_size=batch_size,shuffle=False,num_workers=num_workers)
    
    # train model
    model = model_struct().to('cuda')
    criterion = nn.SmoothL1Loss()
    optimizer = RAdam(model.parameters(),lr=0.0001,weight_decay=1e-2)
    scheduler = ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=5)
    model = train_cnn(model,optimizer,train_dl,valid_dl,n_epochs,clip,scheduler)
    
    # predict oof
    model.eval()
    yhat_list = []
    with torch.no_grad():
        for x_torch,_ in valid_dl:
            x_torch = x_torch.to('cuda:0')
            yhat_list.append(model(x_torch).squeeze(1))
    yhat = torch.cat(yhat_list).cpu().detach().numpy()        
    
    assert yhat.shape[0]==train_ids.iloc[valid_idx].shape[0],'yhat and test_id should have same shape'
    submit_ = dict(zip(train_ids.iloc[valid_idx]['id'].values,yhat))
    train_df['fold'+str(n_fold)+'_'+type_] = train_df.id.map(submit_)
    
    # predict test
    model.eval()
    yhat_list = []
    with torch.no_grad():
        for data_torch in test_dl:
            data_torch = data_torch.to('cuda:0')
            yhat_list.append(model(data_torch).squeeze(1))
    yhat = torch.cat(yhat_list).cpu().detach().numpy()        

    # join
    assert yhat.shape[0]==test_ids.shape[0],'yhat and test_id should have same shape'
    submit_ = dict(zip(test_ids['id'].values,yhat))
    test_df['fold'+str(n_fold)+'_'+type_] = test_df.id.map(submit_)    


start fold: 0
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Epoch: 1 	Training Loss: 2.879799 	Validation Loss: 2.860868
Epoch: 2 	Training Loss: 2.849440 	Validation Loss: 2.826412
Training completed in 39.659775257110596s
